In [1]:
library(plyr, quietly = TRUE)
library(tidyverse, quietly = TRUE)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()   masks plyr::arrange()
✖ purrr::compact()   masks plyr::compact()
✖ dplyr::count()     masks plyr::count()
✖ dplyr::desc()      masks plyr::desc()
✖ dplyr::failwith()  masks plyr::failwith()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::id()        masks plyr::id()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::mutate()    masks plyr::mutate()
✖ dplyr::rename()    masks plyr::rename()
✖ dplyr::summarise() masks plyr::summarise()
✖ dplyr::summarize() masks plyr::summarize()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
LLM_score_thresh = 0.8
enrichr_adj_pval_thresh = 0.05
enrichr_JI_thresh = 0.2

In [3]:
reduced_LLM_genes_DF = read_delim(file = "data/omics_test_revamped_LLM_Enrichr_simVals_DF.tsv", delim = "\t")

New names:
• `` -> `...1`
Rows: 40 Columns: 18
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (10): Source, GeneSetID, GeneSetName, GeneList, LLM Name, LLM Analysis, ...
dbl  (8): ...1, index, n_Genes, Score, Rank, P-value, Adjusted P-value, LLM_...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
get_JI = function(GeneList, enrichr_genes){
    # assume separation is the same
    geneSetGenes = str_split(string = GeneList, pattern = " ")[[1]]
    enrichRGenes =  str_split(string = enrichr_genes, pattern = ";")[[1]]
    JI = length(intersect(geneSetGenes, enrichRGenes))/ length(union(geneSetGenes, enrichRGenes))
    
    return(JI)
    }
    

In [5]:
reduced_LLM_genes_DF = reduced_LLM_genes_DF %>%
rowwise() %>%
mutate(enrichr_JI = get_JI(GeneList, Genes)) %>%
mutate(LLM_success_TF = ifelse(Score < 0.8, FALSE, TRUE),
       enrichr_success_TF = ifelse(((`Adjusted P-value` > enrichr_adj_pval_thresh) | (enrichr_JI < enrichr_JI_thresh)), FALSE, TRUE))

In [6]:
table(reduced_LLM_genes_DF[,c("LLM_success_TF", "enrichr_success_TF")])

              enrichr_success_TF
LLM_success_TF FALSE TRUE
         FALSE    11    2
         TRUE     15   12

In [7]:
15/26

[1] 0.5769231

In [10]:
a = reduced_LLM_genes_DF %>%
subset((LLM_success_TF == FALSE) & (enrichr_success_TF == TRUE))

In [23]:
a %>% select(-"LLM Analysis") %>% select(c("GeneSetName", "GeneList", "LLM Name", "Adjusted P-value", "GO term", "enrichr_JI")) %>%
write_delim(file = "data/failure.txt", delim = "\t")

In [ ]:
colnames(reduced_LLM_genes_DF)

In [13]:
both_success_DF = reduced_LLM_genes_DF %>%
subset((LLM_success_TF == TRUE) & (enrichr_success_TF == TRUE))

In [14]:
library(ggpubr)


Attaching package: ‘ggpubr’


The following object is masked from ‘package:plyr’:

    mutate




In [20]:
both_success_DF %>% select(-"LLM Analysis") %>% tail()

...1,index,Source,GeneSetID,GeneSetName,GeneList,n_Genes,LLM Name,Score,Rank,Overlap,P-value,Adjusted P-value,Genes,GO term,GO ID,LLM_name_GO_term_sim,enrichr_JI,LLM_success_TF,enrichr_success_TF
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<lgl>,<lgl>
26,0,NeST,Cluster1-16,Cluster1-16,DPP6 KCND2 KCND3 KCNE4 KCNIP1 KCNIP2 KCNIP4 SRSF3 SUGP2 TARDBP TRNAU1AP,11,Regulation of Cardiac and Neuronal Excitability,0.92,0,4/30,1.346836e-09,7.836523e-08,DPP6;KCNIP1;KCNIP2;KCNIP4,Regulation Of Potassium Ion Transport,GO:0043266,0.4910333,0.3636364,TRUE,TRUE
27,0,NeST,Cluster1-17,Cluster1-17,ATAD5 FANCI PHLPP1 RAD51AP1 USP1 USP12 WDR48,7,DNA Damage Response and Repair,0.92,0,2/36,6.577641e-05,1.381305e-03,FANCI;RAD51AP1,Interstrand Cross-Link Repair,GO:0036297,0.6352661,0.2857143,TRUE,TRUE
28,0,NeST,Cluster1-18,Cluster1-18,AK2 CDH1 HADHA HADHB MT-ATP8 MT-ND4L SQSTM1,7,Mitochondrial Energy Metabolism and Cellular Homeostasis,0.88,0,2/49,1.225173e-04,6.215530e-03,HADHB;HADHA,Fatty Acid Beta-Oxidation,GO:0006635,0.4247301,0.2857143,TRUE,TRUE
29,0,NeST,Cluster1-19,Cluster1-19,EGFR NR3C1 PRDX1 PRDX2 PRDX5 TXN,6,Redox Regulation and Signal Transduction,0.92,0,3/20,1.706727e-08,2.662494e-06,PRDX2;PRDX5;PRDX1,Hydrogen Peroxide Catabolic Process,GO:0042744,0.3988067,0.5000000,TRUE,TRUE
37,0,Small Molecules,doxorubicin_homo sapiens_gpl7168_gse12450,doxorubicin homo sapiens gpl7168 gse12450,CD3D ANKRD33B IL1B ABHD1,4,Immune Response Modulation,0.85,0,1/5,9.996772e-04,1.497744e-02,IL1B,Regulation Of Immature T Cell Proliferation In Thymus,GO:0033084,0.4898155,0.2500000,TRUE,TRUE
39,0,Small Molecules,estradiol_homo sapiens_gpl91_gds884,estradiol homo sapiens gpl91 gds884,AURKA CFL1 TUBB UBC HSP90AB1 ENO1 HSPB1 ACTB ACTG1 B2M GAPDH RPL10 HLA-C CLU RPL24 RPS15 RPS2 RPS17 RPS9 RPS3 RPL9 RPS12 RPS23 RPS3A RPLP1 RPS19 RPL8 RPL32 RPL17 RPS16 RPS21 HMGN2 RPL37A RPS20 TMSB4X GNAS ANXA2P3 RPSAP19 ANKRD12 FOXJ2 RHOBTB1 DDX5 ARF1 CANX SYPL1 HIF1A VEGFC TXNRD2 SERTAD2 KIF3B SEC16A COPB1 EIF2AK2 CAPN2 JAG1 HSPG2 AHNAK SND1 FEZ2 PLK2 DSTN CSRP1 OAT KIF2C NDUFS6 SFI1 NUDC PTP4A1 ILF3 EIF4G1 RPS6KA1 RND3 SAMD4A MACF1 MCL1 YIF1A SSR2 ZBTB5 AP1S1 TXNRD1 MYL12B HSPA8 ALDOA DDIT4 PLIN2 SFN CRIM1 COX7A2 APP RPL35 RPL13A RPL23 TOMM20 CDK2AP2 KDELR1 CITED2 MAGEA5 MT1P3 LRP3 RPL21P80,100,Cellular Structural Integrity and Protein Synthesis Regulation,0.88,0,21/93,1.412132e-29,1.436138e-26,RPS9;RPL32;RPL10;RPLP1;RPL13A;RPS3A;RPL8;RPL9;RPS15;RPS17;RPS16;RPS19;RPL37A;RPS3;RPL35;RPL24;RPS20;RPS2;RPL17;RPS21;RPS23,Cytoplasmic Translation,GO:0002181,0.4613842,0.2100000,TRUE,TRUE


In [24]:
hist_plot = gghistogram(data = both_success_DF, x = "LLM_name_GO_term_sim", y = "count", bins = 100)

In [25]:
ggsave(filename = "figures/omics_revamped_test_hist_sim_LLM_GO_success.png", width = 5, height = 2)

In [30]:
both_success_DF %>% subset(LLM_name_GO_term_sim > 0.8) %>% select(-"LLM Analysis") %>%
select(c("GeneSetName", "GeneList", "LLM Name", "Adjusted P-value", "GO term", "enrichr_JI"))  %>%
write_delim(file = "data/both_success_high_sim.txt", delim = "\t")

In [31]:
both_success_DF %>% subset(LLM_name_GO_term_sim < 0.3) %>% select(-"LLM Analysis") %>%
select(c("GeneSetName", "GeneList", "LLM Name", "Adjusted P-value", "GO term", "enrichr_JI"))  %>%
write_delim(file = "data/both_success_low_sim.txt", delim = "\t")

In [ ]:
distribution of semantic similarty for both success

In [ ]:
similarity between LLM name and enrichr name for both success